AlexNet trained using Transfer Learning (Fine Tunning)
------------
Training AlexNet model on Oxford-IIIT dataset using a variant of Transfer Learning.

Parameters of all layers are slowly updated (using smaller learning rate).

Notes:
- Notebook expects that you have downloaded the dataset into your drive.
- Change the path to reflect your location.

In [1]:
from google.colab import drive

#mount the drive
MOUNTING_LOCATION = '/content/drive'
print('Mounting the drive...')
drive.mount(MOUNTING_LOCATION)
print(f"Drive mounted at: {MOUNTING_LOCATION}")

#Unzip the train and validation datasets
print('Unzipping the train dataset...')
!unzip -q "/content/drive/My Drive/DIPLOMSKI/oxford-iiit/train.zip" -d '/content' 
print('Finished unzipping the train dataset. Now unzipping the validation dataset...')
!unzip -q "/content/drive/My Drive/DIPLOMSKI/oxford-iiit/val.zip" -d '/content'
print('Finished unzipping the validation dataset.')


Mounting the drive...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Drive mounted at: /content/drive
Unzipping the train dataset...
replace /content/train/Abyssinian/Abyssinian_1.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: N
Finished unzipping the train dataset. Now unzipping the validation dataset...
replace /content/val/Abyssinian/Abyssinian_112.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: N
Finished unzipping the validation dataset.


In [2]:
import torch
from PIL import Image
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import time
import os
import json
import copy

dataset_props = {'DATA_DIR': '/content/',
                 'N_CLASSES': 37
                 }

MODEL_NAME = 'A2'
model_props = {'MODEL_NAME': MODEL_NAME,
               'PRETRAINED': True,
               'SAVE_PATH': f'/content/drive/My Drive/DIPLOMSKI/{MODEL_NAME}/',
               'SAVE_EVERY_N_EPOCHS': 5,
               'TRAIN_NAME_SAVE': f'{MODEL_NAME}_train_model.pt',
               'VAL_NAME_SAVE': f'{MODEL_NAME}_val_model.pt',
               'LOG_NAME': f'{MODEL_NAME}_log.json',
               'INPUT_SIZE': 224,
               'N_EPOCHS': 1000,
               'BATCH_SIZE': 64,
               'LEARNING_RATE': 1e-6,
               'MOMENTUM': 0.9,
               'DEVICE': torch.device("cuda:0" if torch.cuda.is_available() else "cpu")}

#transforms to be applied to images before them being fed to the model
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(model_props['INPUT_SIZE']),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(model_props['INPUT_SIZE']),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}
#building the dataset loaders
image_datasets = {x: datasets.ImageFolder(os.path.join(dataset_props['DATA_DIR'], x), 
                                          data_transforms[x]) for x in ['train', 'val']}
dataloaders_dict = {x: torch.utils.data.DataLoader(image_datasets[x],
                                                   batch_size=model_props['BATCH_SIZE'],
                                                   shuffle=True) for x in ['train', 'val']}



In [3]:

#Build the Model
model = torch.hub.load('pytorch/vision:v0.6.0', 'alexnet', pretrained=model_props['PRETRAINED'])

#freeze all parameters
for param in model.parameters():
  param.requires_grad = False

#Replace the last fullyconnected layer - to match the number of classes in this dataset
num_ftrs = model.classifier[6].in_features
model.classifier[6] = nn.Linear(num_ftrs, dataset_props['N_CLASSES'])

#put the model inside GPU memory if GPU is available
if model_props['DEVICE'] == torch.device('cuda:0'): 
  model.cuda()

#make a list of all the parameters that will get optimized (needed when using transfer learning)
optimize_params = [x for x in model.parameters() if x.requires_grad == True]

#define the loss function and the optimizer
criterion = nn.CrossEntropyLoss()
if model_props['DEVICE'] == torch.device('cuda:0'): 
  criterion.cuda()
optimizer = optim.SGD(optimize_params,
                      lr = model_props['LEARNING_RATE'], 
                      momentum = model_props['MOMENTUM'])


Downloading: "https://github.com/pytorch/vision/archive/v0.6.0.zip" to /root/.cache/torch/hub/v0.6.0.zip
Downloading: "https://download.pytorch.org/models/alexnet-owt-4df8aa71.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-4df8aa71.pth


In [4]:
#training and evaluating the model
best_val_acc = -1.0
for epoch in range(model_props['N_EPOCHS']):

  phase_loss = {'train': 0.0, 'val': 0.0}; phase_acc = {'train': 0.0, 'val': 0.0}

  #Pytorch uses two phase system
  for phase in ['train', 'val']:
    if phase == 'train':
      model.train()  #model learns
    else:
      model.eval()   #model is just being evaluated

    for inputs, labels in dataloaders_dict[phase]:
      #moving data to the GPU - if available
      inputs = inputs.to(model_props['DEVICE']) ; labels = labels.to(model_props['DEVICE'])

      #gradients are being added - thus this line is needed
      optimizer.zero_grad()
      #gradient being calculate only if it is the training phase
      with torch.set_grad_enabled(phase == 'train'):

        outputs = model(inputs) ; loss = criterion(outputs, labels)

        _, preds = torch.max(outputs, 1)

        #backprop only if it is the training phase
        if phase == 'train':
          loss.backward()
          optimizer.step()

      #noting the progress
      phase_loss[phase] += loss.item() * inputs.size(0)
      phase_acc[phase] += torch.sum(preds == labels.data)

    phase_loss[phase] = (phase_loss[phase] / len(dataloaders_dict[phase].dataset))
    phase_acc[phase] = (phase_acc[phase] / len(dataloaders_dict[phase].dataset)).item()
    
    #Saving the current model
    if phase == 'train' and epoch % model_props['SAVE_EVERY_N_EPOCHS']:
      torch.save(model.state_dict(), f"{model_props['SAVE_PATH']}{model_props['TRAIN_NAME_SAVE']}")

    #keep the best model
    if phase == 'val' and best_val_acc < phase_acc[phase]:
      torch.save(model.state_dict(), f"{model_props['SAVE_PATH']}{model_props['VAL_NAME_SAVE']}")
      best_val_acc = phase_acc[phase]


  #append to the log file - to keep information of the progress through epochs
  try:
    with open(f"{model_props['SAVE_PATH']}{model_props['LOG_NAME']}") as f:
      data = json.load(f)
  except:
    data = {'epoch':0,
            'loss': [], 
            'acc': []}
            
  data['best_val_acc'] = best_val_acc
  data['epoch'] += 1
  with open(f"{model_props['SAVE_PATH']}{model_props['LOG_NAME']}", 'w') as f:
    data['loss'].append(phase_loss)
    data['acc'].append(phase_acc)
    json.dump(data, f)

  #print out the progress information
  print(f"EPOCH: {data['epoch']}/{data['epoch'] + model_props['N_EPOCHS']} --------------------------------------------")
  print(f"\tTrain phase: Loss: {phase_loss['train']} ; Acc: {phase_acc['train']}")
  print(f"\tVal phase: Loss: {phase_loss['val']} ; Acc: {phase_acc['val']}")


KeyboardInterrupt: ignored